Pytree nodes interlude

We come to an issue here of what fields should we tell jax to ignore (i.e. set pytree_node=False). The TrainState for flax only tells jax to ignore the functions (tx and apply_fn). Variables like iteration, epoch, and seed don't need their gradient being taken. They will also most likely not need to be vmapped over (we shouldn't have the need to vectorize over any of these). The other transformation would be jitting. I'm not sure what the implications of jitting a function with pytree_nodes set to false. Let's explore this.

In [22]:
from jax import numpy as jnp


@struct.dataclass
class Bag:
    a: Any
    b: Any
    
    
def sum_bag(bag):
    return bag.a * bag.a + bag.b * 2.0

a = jnp.ones((5000, 5000))

b = 2.0 * jnp.ones((5000, 5000))

bag = Bag(a, b)
sum_bag(bag)

DeviceArray([[5., 5., 5., ..., 5., 5., 5.],
             [5., 5., 5., ..., 5., 5., 5.],
             [5., 5., 5., ..., 5., 5., 5.],
             ...,
             [5., 5., 5., ..., 5., 5., 5.],
             [5., 5., 5., ..., 5., 5., 5.],
             [5., 5., 5., ..., 5., 5., 5.]], dtype=float32)

In [23]:
%timeit sum_bag(bag)

859 µs ± 393 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [60]:
from jax import jit, vmap

jitted_sum_bag = jit(sum_bag)
jitted_sum_bag(bag)

Bag(a=DeviceArray([[1., 1., 1., ..., 1., 1., 1.],
             [1., 1., 1., ..., 1., 1., 1.],
             [1., 1., 1., ..., 1., 1., 1.],
             ...,
             [1., 1., 1., ..., 1., 1., 1.],
             [1., 1., 1., ..., 1., 1., 1.],
             [1., 1., 1., ..., 1., 1., 1.]], dtype=float32), b=DeviceArray([[2., 2., 2., ..., 2., 2., 2.],
             [2., 2., 2., ..., 2., 2., 2.],
             [2., 2., 2., ..., 2., 2., 2.],
             ...,
             [2., 2., 2., ..., 2., 2., 2.],
             [2., 2., 2., ..., 2., 2., 2.],
             [2., 2., 2., ..., 2., 2., 2.]], dtype=float32), c=DeviceArray([[5., 5., 5., ..., 5., 5., 5.],
             [5., 5., 5., ..., 5., 5., 5.],
             [5., 5., 5., ..., 5., 5., 5.],
             ...,
             [5., 5., 5., ..., 5., 5., 5.],
             [5., 5., 5., ..., 5., 5., 5.],
             [5., 5., 5., ..., 5., 5., 5.]], dtype=float32))

In [36]:
@struct.dataclass
class Bag:
    a: Any
    b: Any
    
    
bag = Bag(a, b)

%timeit jitted_sum_bag(bag)

362 µs ± 90.8 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [38]:
@struct.dataclass
class Bag:
    a: Any
    b: Any = struct.field(pytree_node=False)
    

bag = Bag(a, b)

%timeit jitted_sum_bag(bag)

85.3 µs ± 28.5 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [40]:
@struct.dataclass
class Bag:
    a: Any = struct.field(pytree_node=False)
    b: Any = struct.field(pytree_node=False)
    

bag = Bag(a, b)

%timeit jitted_sum_bag(bag)

38.7 µs ± 18.6 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [44]:
def sum_bag(bag):
    return bag.replace(c=bag.a * bag.a + bag.b * 2.0)

jitted_sum_bag = jit(sum_bag)

In [46]:
@struct.dataclass
class Bag:
    a: Any
    b: Any
    c: Any = None
    
    
bag = Bag(a, b)

%timeit jitted_sum_bag(bag)

844 µs ± 36.7 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [48]:
@struct.dataclass
class Bag:
    a: Any
    b: Any
    c: Any = struct.field(pytree_node=False, default=None)
    
    
bag = Bag(a, b)

%timeit jitted_sum_bag(bag)

484 µs ± 33.4 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [55]:
@struct.dataclass
class Bag:
    a: Any
    b: Any
    c: Any = struct.field(pytree_node=False, default=tuple())
    
    
def change_bag(bag):
    return bag.replace(c=2.0)


jitted_change_bag = jit(change_bag)
    
    
bag = Bag(a, b)

jitted_change_bag(bag)

Bag(a=DeviceArray([[1., 1., 1., ..., 1., 1., 1.],
             [1., 1., 1., ..., 1., 1., 1.],
             [1., 1., 1., ..., 1., 1., 1.],
             ...,
             [1., 1., 1., ..., 1., 1., 1.],
             [1., 1., 1., ..., 1., 1., 1.],
             [1., 1., 1., ..., 1., 1., 1.]], dtype=float32), b=DeviceArray([[2., 2., 2., ..., 2., 2., 2.],
             [2., 2., 2., ..., 2., 2., 2.],
             [2., 2., 2., ..., 2., 2., 2.],
             ...,
             [2., 2., 2., ..., 2., 2., 2.],
             [2., 2., 2., ..., 2., 2., 2.],
             [2., 2., 2., ..., 2., 2., 2.]], dtype=float32), c=2.0)

In [59]:
@struct.dataclass
class Bag:
    a: Any
    b: Any
    c: Any = struct.field(pytree_node=True, default=tuple())
    
    
def change_bag(bag):
    return bag.replace(c=2.0)


jitted_change_bag = jit(change_bag)
    
    
bag = Bag(a, b)

jitted_change_bag(bag)

Bag(a=DeviceArray([[1., 1., 1., ..., 1., 1., 1.],
             [1., 1., 1., ..., 1., 1., 1.],
             [1., 1., 1., ..., 1., 1., 1.],
             ...,
             [1., 1., 1., ..., 1., 1., 1.],
             [1., 1., 1., ..., 1., 1., 1.],
             [1., 1., 1., ..., 1., 1., 1.]], dtype=float32), b=DeviceArray([[2., 2., 2., ..., 2., 2., 2.],
             [2., 2., 2., ..., 2., 2., 2.],
             [2., 2., 2., ..., 2., 2., 2.],
             ...,
             [2., 2., 2., ..., 2., 2., 2.],
             [2., 2., 2., ..., 2., 2., 2.],
             [2., 2., 2., ..., 2., 2., 2.]], dtype=float32), c=DeviceArray(2., dtype=float32, weak_type=True))

We can also ask what it means to vmap over a dataclass

In [82]:
@struct.dataclass
class Bag:
    a: Any
    b: Any
    c: Any = struct.field(pytree_node=False, default=tuple())
    

def change_bag(bag):
    return bag.replace(a=bag.a * bag.a, c=2.0)
    
vmapped_change_bag = vmap(change_bag)

bag = Bag(a, b)

res = vmapped_change_bag(bag)
print(res.a.shape, res.b.shape)
res

(5000, 5000) (5000, 5000)


Bag(a=DeviceArray([[1., 1., 1., ..., 1., 1., 1.],
             [1., 1., 1., ..., 1., 1., 1.],
             [1., 1., 1., ..., 1., 1., 1.],
             ...,
             [1., 1., 1., ..., 1., 1., 1.],
             [1., 1., 1., ..., 1., 1., 1.],
             [1., 1., 1., ..., 1., 1., 1.]], dtype=float32), b=DeviceArray([[2., 2., 2., ..., 2., 2., 2.],
             [2., 2., 2., ..., 2., 2., 2.],
             [2., 2., 2., ..., 2., 2., 2.],
             ...,
             [2., 2., 2., ..., 2., 2., 2.],
             [2., 2., 2., ..., 2., 2., 2.],
             [2., 2., 2., ..., 2., 2., 2.]], dtype=float32), c=2.0)

In [83]:
@struct.dataclass
class Bag:
    a: Any
    b: Any
    c: Any = struct.field(pytree_node=True, default=tuple())
    

vmapped_change_bag = vmap(change_bag)

bag = Bag(a, b)

res = vmapped_change_bag(bag)
print(res.a.shape, res.b.shape, res.c.shape)
res

(5000, 5000) (5000, 5000) (5000,)


Bag(a=DeviceArray([[1., 1., 1., ..., 1., 1., 1.],
             [1., 1., 1., ..., 1., 1., 1.],
             [1., 1., 1., ..., 1., 1., 1.],
             ...,
             [1., 1., 1., ..., 1., 1., 1.],
             [1., 1., 1., ..., 1., 1., 1.],
             [1., 1., 1., ..., 1., 1., 1.]], dtype=float32), b=DeviceArray([[2., 2., 2., ..., 2., 2., 2.],
             [2., 2., 2., ..., 2., 2., 2.],
             [2., 2., 2., ..., 2., 2., 2.],
             ...,
             [2., 2., 2., ..., 2., 2., 2.],
             [2., 2., 2., ..., 2., 2., 2.],
             [2., 2., 2., ..., 2., 2., 2.]], dtype=float32), c=DeviceArray([2., 2., 2., ..., 2., 2., 2.], dtype=float32, weak_type=True))

It looks like we can concude that any field we mark are not a pytree node will make the function faster. This makes sense because we give additional functionality to nodes marked as pytree nodes